# Setup

Switch kernel to bw25 if necessary

In [1]:
import bw_processing as bwp
import matrix_utils as mu
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import numpy as np
import seaborn as sb
import pandas as pd
from pathlib import Path

In [129]:
import bw2analyzer as ba

In [4]:
if 'ei38-teaching-25' not in bd.projects:
    bi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")

In [5]:
bd.projects # dont know if we need to create our own project?

Brightway2 projects manager with 9 objects:
	Correlated and dependent sampling
	Lithium GSA
	brightcon22
	datapackage demo
	default
	ei38-teaching
	ei38-teaching-25
	ei_38
	your project name
Use `projects.report()` to get a report on all projects.

In [53]:
bd.projects.set_current('ei38-teaching-25')

In [54]:
bd.databases

Databases dictionary with 4 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	biosphere3
	ei 3.8 cutoff

# Create activities?

In [55]:
# easy to change names of biosphere and ei db:s
biosphere = 'biosphere3'
ecoinvent = 'ei 3.8 cutoff'

In [56]:
# could be good to have idk
eidb = bd.Database('ei 3.8 cutoff')
my_bio = bd.Database(biosphere)

## Template for searching activities etc

In [ ]:
bd.Database(biosphere).search('carbon dioxide')

In [ ]:
bd.Database(ecoinvent).search('copper')

In [31]:
activity_I_want = [act for act in bd.Database(biosphere) if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'non' not in act['name']
                                            and 'urban air close to ground' in str(act['categories'])
         ][0]
activity_I_want

'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))

In [32]:
activity_I_want.key

('biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9')

In [34]:
bd.get_activity(activity_I_want.key)

'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))

In [51]:
bd.Database(biosphere).search('carbon dioxide', filter={'categories':'urban', 'name':'fossil'})

Excluding 19 filtered results


['Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground'))]

In [ ]:
# Using search
bd.Database(ecoinvent).search('transport', filter={'name':'lorry'})

## Methods

In [39]:
ipcc2013 = [m for m in bd.methods if 'IPCC' in m[0]
                    and ('2013') in str(m)
                    and 'GWP 100' in str(m)
                    and 'no LT' not in str(m)][0]

In [42]:
type(ipcc2013)

tuple

In [40]:
ipcc_2013_method = bd.Method(ipcc2013)

In [41]:
type(ipcc_2013_method)

bw2data.method.Method

In [41]:
ipcc_2013_method.name

('IPCC 2013', 'climate change', 'GWP 100a')

In [42]:
ipcc_2013_method.metadata

{'description': "IPCC characterisation factors for the direct global warming potential of air emissions in ecoinvent 3.2. See the ecoinvent report 'Implementation of IPCC impact assessment method 2007 and 2013 to ecoinvent database 3.2 (2015.11.30)' for more details. Doesn't include: indirect formation of dinitrogen monoxide from nitrogen emissions, radiative forcing due to emissions of NOx, water, sulphate, etc. in the lower stratosphere + upper troposphere, carbon climate feedbacks, and any emissions which have cooling effects (e.g. black carbon). All CO is assumed to convert completely to CO2. Biogenic CO2 uptake and biogenic CO2 emissions are not characterised, except for 'Carbon dioxide, from soil or biomass stock' (deforestation and land transformation).",
 'filename': 'LCIA_Implementation_3.8.xlsx',
 'unit': 'kg CO2-Eq',
 'abbreviation': 'ipcc-2013cg.bd5af3f67229a1cc291b8ecb7f316fcf',
 'num_cfs': 211}

In [43]:
ipcc_2013_method.metadata['unit']

'kg CO2-Eq'

# Creating LCI

In [74]:
# creating new activity? let's do that in our database?
bd.databases

Databases dictionary with 5 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	biosphere3
	charcrete
	ei 3.8 cutoff

In [73]:
user_db = bd.Database("charcrete")

In [70]:
bd.databases

Databases dictionary with 4 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	biosphere3
	ei 3.8 cutoff

In [84]:
example_data3 = {
    ("charcrete", "A"): {
        "name": "A",
        "exchanges": [{
            "amount": 1.0,
            "input": ("charcrete", "B"),
            "type": "technosphere"
            }],
        'unit': 'kilogram',
        'location': 'here',
        'categories': ("very", "interesting")
        },
    ("charcrete", "B"): {
        "name": "B",
        "exchanges": [],
        'unit': 'microgram',
        'location': 'there',
        'categories': ('quite', 'boring')
        }
    }

In [85]:
user_db.write(example_data3)

Not able to determine geocollections for all datasets. This database is not ready for regionalization.
Title: Writing activities to SQLite3 database:
  Started: 10/26/2022 12:16:34
  Finished: 10/26/2022 12:16:34
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 0.20


In [87]:
len(user_db)

2

In [109]:
user_db.search('*')

[]

In [104]:
act = user_db.search('*')[0]

In [107]:
del(act)

In [108]:
user_db.search('*')

[]

In [110]:
user_db

Brightway2 SQLiteBackend: charcrete

In [111]:
len(user_db)

0

## find data

In [112]:
bd.Database(ecoinvent).search('forwarding')

['market for forwarding, forwarder' (hour, GLO, None),
 'forwarding, forwarder' (hour, RER, None),
 'forwarding, forwarder' (hour, RoW, None)]

In [115]:
activity_I_want = [act for act in bd.Database(ecoinvent) if 'forwarding' in act['name'] and act['location'] == 
                   'RER'
         ][0]
activity_I_want

'forwarding, forwarder' (hour, RER, None)

In [116]:
forwarding_act = [act for act in bd.Database(ecoinvent) if 'forwarding' in act['name'] and act['location'] == 
                   'RER'][0]

In [117]:
forwarding_act

'forwarding, forwarder' (hour, RER, None)

In [118]:
bd.Database(ecoinvent).search('wood chipping, mobile chipper, at forest road')

['market for wood chipping, chipper, mobile, diesel, at forest road' (hour, GLO, None),
 'wood chipping, mobile chipper, at forest road' (hour, RER, None),
 'wood chipping, mobile chipper, at forest road' (hour, RoW, None)]

In [119]:
activity_I_want = [act for act in bd.Database(ecoinvent) if 'wood chipping, mobile chipper, at forest road' in act['name'] and act['location'] == 
                   'RER'
         ][0]
activity_I_want

'wood chipping, mobile chipper, at forest road' (hour, RER, None)

In [120]:
activity_I_want

'wood chipping, mobile chipper, at forest road' (hour, RER, None)

In [121]:
wood_chipping_act = [act for act in bd.Database(ecoinvent) if 'wood chipping, mobile chipper, at forest road' in act['name'] 
                     and act['location'] == 'RER'][0]

In [122]:
wood_chipping_act

'wood chipping, mobile chipper, at forest road' (hour, RER, None)

# regionalization of concrete production?

In [134]:
concrete_market = bd.get_activity(database=ecoinvent, name="market group for concrete, normal", location = "GLO")

In [125]:
concrete_market

'market group for concrete, normal' (cubic meter, GLO, None)

In [126]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

In [131]:
ba.print_recursive_calculation(concrete_market, ipcc, cutoff=0.001) # CA
# short cut: we assume that the ones contributing most is the ones we need to regionalize
# is there a way to sort this data?

Fraction of score | Absolute score | Amount | Activity
0001 | 283.4 |     1 | 'market group for concrete, normal' (cubic meter, GLO, None)
  0.0144 | 4.086 | 0.0226 | 'market for concrete, normal' (cubic meter, BR, None)
    0.0144 | 4.086 | 0.0226 | 'concrete, all types to generic market for concrete, normal strength' 
      0.00275 | 0.7798 | 0.003616 | 'market for concrete, 30MPa' (cubic meter, BR, None)
      0.0115 | 3.253 | 0.01876 | 'market for concrete, 25MPa' (cubic meter, BR, None)
  0.00208 | 0.5893 | 0.002355 | 'market for concrete, normal' (cubic meter, CO, None)
    0.00208 | 0.5893 | 0.002355 | 'concrete, all types to generic market for concrete, normal strength' 
      0.00208 | 0.5893 | 0.002355 | 'market for concrete, 20MPa' (cubic meter, CO, None)
  0.0261 |  7.39 | 0.01907 | 'market for concrete, normal' (cubic meter, IN, None)
    0.0261 |  7.39 | 0.01907 | 'concrete, all types to generic market for concrete, normal strength' 
      0.0261 |  7.39 | 0.01907 | 'mark

In [138]:
# could also do specifically for 20MPa
concrete_markets = [act for act in bd.Database(ecoinvent) if 'market for concrete, 20MPa' in act['name']]

In [139]:
concrete_markets

['market for concrete, 20MPa' (cubic meter, PE, None),
 'market for concrete, 20MPa' (cubic meter, ZA, None),
 'market for concrete, 20MPa' (cubic meter, RNA, None),
 'market for concrete, 20MPa' (cubic meter, RoW, None),
 'market for concrete, 20MPa' (cubic meter, CO, None)]

PE = Peru

ZA = South Africa

RNA = Northern America

RoW = Rest of world

CO = Colombia

In [140]:
fu, data_objs, _ = bd.prepare_lca_inputs({concrete_market: 1}, method=ipcc)

In [141]:
lca = bc.LCA(fu, data_objs=data_objs)

In [142]:
lca.lci()
lca.lcia()

In [144]:
lca.score # global market for "normal" concrete

283.44576866175254

In [150]:
concrete_markets[0]['location']

'PE'

In [151]:
# calculate score for different markets
for market in concrete_markets:
    fu, data_objs, _ = bd.prepare_lca_inputs({market: 1}, method=ipcc)
    lca = bc.LCA(fu, data_objs=data_objs)
    lca.lci()
    lca.lcia()
    print(market['location'], lca.score)

PE 341.7487287312562
ZA 234.01566863447212
RNA 246.39245279678855
RoW 237.44010359161643
CO 250.2176994748129


In [152]:
# but then we could also open these markets up? and see the subactivities for prod of concrete
ba.print_recursive_calculation(concrete_markets[0], ipcc, cutoff=0.001) # CA
# short cut: we assume that the ones contributing most is the ones we need to regionalize
# is there a way to sort this data?

Fraction of score | Absolute score | Amount | Activity
0001 | 341.7 |     1 | 'market for concrete, 20MPa' (cubic meter, PE, None)
  0.0199 | 6.805 |    72 | 'market for transport, freight, lorry >32 metric ton, EURO3' (ton kilo
    0.0199 | 6.805 |    72 | 'transport, freight, lorry >32 metric ton, EURO3' (ton kilometer, RoW,
      0.00291 | 0.9936 | 0.07848 | 'market for road' (meter-year, GLO, None)
      0.00204 | 0.6984 |  1.23 | 'market for diesel, low-sulfur' (kilogram, RoW, None)
  0.00887 | 3.032 |  0.01 | 'concrete production, 20MPa, ready-mix, with cement, limestone 21-35%'
    0.00774 | 2.644 | 3.914 | 'market for cement, limestone 21-35%' (kilogram, PE, None)
      0.00747 | 2.553 | 3.914 | 'cement production, limestone 21-35%' (kilogram, PE, None)
  0.00682 | 2.332 | 0.01048 | 'concrete production, 20MPa, ready-mix, with cement, pozzolana and fly
    0.00569 | 1.945 | 3.237 | 'market for cement, pozzolana and fly ash 36-55%' (kilogram, RoW, None
      0.00164 | 0.5619 | 0

What would happen in ecoinvent if we were to change the concrete markets?

In [153]:
wood_chipping_act # will use this one for now - we can put in charcrete later

'wood chipping, mobile chipper, at forest road' (hour, RER, None)

Ok so let's go back to the market for conrete, normal and assume that we change one share of the market to our charcrete activity 

In [154]:
ba.print_recursive_calculation(concrete_market, ipcc, cutoff=0.001) # CA
# short cut: we assume that the ones contributing most is the ones we need to regionalize
# is there a way to sort this data?

Fraction of score | Absolute score | Amount | Activity
0001 | 283.4 |     1 | 'market group for concrete, normal' (cubic meter, GLO, None)
  0.0144 | 4.086 | 0.0226 | 'market for concrete, normal' (cubic meter, BR, None)
    0.0144 | 4.086 | 0.0226 | 'concrete, all types to generic market for concrete, normal strength' 
      0.00275 | 0.7798 | 0.003616 | 'market for concrete, 30MPa' (cubic meter, BR, None)
      0.0115 | 3.253 | 0.01876 | 'market for concrete, 25MPa' (cubic meter, BR, None)
  0.00208 | 0.5893 | 0.002355 | 'market for concrete, normal' (cubic meter, CO, None)
    0.00208 | 0.5893 | 0.002355 | 'concrete, all types to generic market for concrete, normal strength' 
      0.00208 | 0.5893 | 0.002355 | 'market for concrete, 20MPa' (cubic meter, CO, None)
  0.0261 |  7.39 | 0.01907 | 'market for concrete, normal' (cubic meter, IN, None)
    0.0261 |  7.39 | 0.01907 | 'concrete, all types to generic market for concrete, normal strength' 
      0.0261 |  7.39 | 0.01907 | 'mark

So basically, this market is in itself 8 other markets: 

CH

BR

CO

IN

PE

RNA

ZA

RoW

Check that they all add up to 1?

In [155]:
sum(exc['amount'] for exc in concrete_market.technosphere()) # does all inputs sum up to one?

1.0000000000000002

idk why it's not exactly one but ok

In [157]:
for exc in concrete_market.technosphere():
    lca.redo_lcia({exc.input.id: 1})
    print(lca.score, exc.input)
    # print the different scores of the different markets i guess

180.79326949989576 'market for concrete, normal' (cubic meter, BR, None)
145.4238269670096 'market for concrete, normal' (cubic meter, CH, None)
250.21769947481295 'market for concrete, normal' (cubic meter, CO, None)
387.578860852761 'market for concrete, normal' (cubic meter, IN, None)
341.74872873125537 'market for concrete, normal' (cubic meter, PE, None)
285.4110569349046 'market for concrete, normal' (cubic meter, RNA, None)
292.9779465046128 'market for concrete, normal' (cubic meter, ZA, None)
283.1982537946509 'market for concrete, normal' (cubic meter, RoW, None)


ok let's start with a weird example: say that we replace all concrete in RNA with charcrete (wood_chipping_act for now)

In [ ]:
concrete_market.copy(Database="make stuff up") # something on gh to show hpw to build a cupply chain in python idk

In [161]:
# ... can we make this into a generic function? like, given an activity, a subactivity and a replacement activity?

# also - can we visualize a before - after?

market_share = 0.0457 # demand of market (market share)
gangnam_style = 1_000_000 # need a large number because somehting about conflicts in id:s in db - dont want to overlap stuff

indices = np.array(
    [
        (gangnam_style, gangnam_style), # Production exchange for new motor ( activity produces itself ) - need to have this to make matrix square
        (concrete_market.id, gangnam_style),  # the whole market
        (bd.get_activity(database=ecoinvent, name='market for concrete, normal', location = 'RNA').id, gangnam_style), # subtract RNA concrete prod
    ] + [
        (node.id, gangnam_style) for node in [wood_chipping_act] # replace with this - something about new dps replacing old ones - could also have the values being added together?
        # see gh bw processing -> policies
        # we have not changes ei - we have added a new activity
        # in these positions - one new row, one new col
    ], dtype=bwp.INDICES_DTYPE
)
data = np.array([
        1,
        1,
        market_share, # old market
    ] + [
        market_share # new market - same amount
    ]
) 
flip = np.array(
    [False, True, False] + [True for _ in [wood_chipping_act]] # First False because ?, True - motor is consumed, False because numbers are negative ... ?, 
) # could alsos set minussign if you prefer but i still wouldnt know where

In [162]:
dp = bwp.create_datapackage()

dp.add_persistent_vector(
    matrix="technosphere_matrix",
    data_array=data,
    indices_array=indices,
    flip_array=flip,
    name="Market without US",
)

In [163]:
_, data_objs, _ = bd.prepare_lca_inputs({concrete_market: 1}, ipcc) # data_objs - still using ei

In [164]:
lca = bc.LCA({motor.id: 1}, data_objs=data_objs + [dp]) # old motor + add [dp] (new motor)
lca.lci()
lca.lcia()
lca.score

283.44576866175174

In [165]:
lca.lcia({gangnam_style: 1}) # new motor
lca.score

283.0965523271845

# Trying something

In [172]:
concrete_RNA = bd.get_activity(database=ecoinvent, name='market for concrete, normal', location = 'RNA')

In [173]:
concrete_RNA

'market for concrete, normal' (cubic meter, RNA, None)

In [185]:
for exc in list(concrete_market.exchanges()):
    print(exc)
    print(exc['name'])
    print(exc.input.id)
    print(concrete_RNA.id)
    if exc.input.id == concrete_RNA.id:
        print('True')
        print(exc['amount'])

Exchange: 1.0 cubic meter 'market group for concrete, normal' (cubic meter, GLO, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>
concrete, normal
18901
22099
Exchange: 0.0226016949011232 cubic meter 'market for concrete, normal' (cubic meter, BR, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>
concrete, normal
5266
22099
Exchange: 0.000680925588410356 cubic meter 'market for concrete, normal' (cubic meter, CH, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>
concrete, normal
18417
22099
Exchange: 0.00235506608812062 cubic meter 'market for concrete, normal' (cubic meter, CO, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>
concrete, normal
11087
22099
Exchange: 0.01906591647549 cubic meter 'market for concrete, normal' (cubic meter, IN, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>
concrete, normal
14553
22099
Exchange: 0.0107245780174631 cubic meter 'market for concrete, 

In [196]:
def replace_stuff(main_act, sub_act, repl_act, method):
    
    # first we need the demand of the sub_act in the main act
    for exc in list(main_act.exchanges()):
        if exc.input.id == sub_act.id:
            demand = exc['amount']
    
    gangnam_style = 1_000_000

    indices = np.array(
    [
        (gangnam_style, gangnam_style), # Production exchange for new main_act ( activity produces itself ) - need to have this to make matrix square
        (main_act.id, gangnam_style),  
        (sub_act.id, gangnam_style), # subtract sub_act
    ] + [
        (node.id, gangnam_style) for node in [repl_act] # replace with this - something about new dps replacing old ones - could also have the values being added together?
        # see gh bw processing -> policies
    ], dtype=bwp.INDICES_DTYPE
    )
    
    data = np.array([
            1,
            1,
            demand, # old activity
        ] + [
            demand # new activity - same amount
        ]
    ) 
    flip = np.array(
        [False, True, False] + [True for _ in [repl_act]] # First False because ?, True - motor is consumed, False because numbers are negative ... ?,
        # WAIT I think I get it. This is the data array, where we put False for old act, and True for new act...
        # ...This is probably where we would want to change things to create new market shares...?
    ) # could alsos set minussign if you prefer but i still wouldnt know where
    
    # return(demand)
    
    dp = bwp.create_datapackage()

    dp.add_persistent_vector(
        matrix="technosphere_matrix",
        data_array=data,
        indices_array=indices,
        flip_array=flip,
        name="New technosphere",
    )
    
    _, data_objs, _ = bd.prepare_lca_inputs({main_act: 1}, ipcc) # data_objs - still using ei
    
    lca = bc.LCA({main_act.id: 1}, data_objs=data_objs + [dp]) # old motor + add [dp] (new motor)
    lca.lci()
    lca.lcia()
    lcascore1 = lca.score # first version
    
    lca.lcia({gangnam_style: 1}) # new motor
    lcascore2 = lca.score # substituted sub_act w repl_act
    
    return(lcascore1, lcascore2)

In [197]:
print(replace_stuff(concrete_market, concrete_RNA, wood_chipping_act, ipcc))

(283.44576866175174, 283.0987257563718)


In [198]:
concrete_market

'market group for concrete, normal' (cubic meter, GLO, None)

In [199]:
concrete_RNA

'market for concrete, normal' (cubic meter, RNA, None)

In [200]:
wood_chipping_act

'wood chipping, mobile chipper, at forest road' (hour, RER, None)

In [201]:
ipcc

('IPCC 2013', 'climate change', 'GWP 100a')

now... it would be supercool if we could visualize this...

The amount of each input and the share of the impacts... before and after...

Let's draw what we want to see

# Trying something - changing market shares

In [ ]:
(node.id, gangnam_style) for node in [repl_act] # this one works...

In [229]:
test=[(exc.input.id) for exc in concrete_market.exchanges()]

In [230]:
test

[18901, 5266, 18417, 11087, 14553, 23135, 22099, 16409, 19248]

In [231]:
test=[(exc.input.id, 1000) for exc in concrete_market.exchanges()]

In [232]:
test

[(18901, 1000),
 (5266, 1000),
 (18417, 1000),
 (11087, 1000),
 (14553, 1000),
 (23135, 1000),
 (22099, 1000),
 (16409, 1000),
 (19248, 1000)]

In [220]:
list(concrete_market.exchanges())

[Exchange: 1.0 cubic meter 'market group for concrete, normal' (cubic meter, GLO, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>,
 Exchange: 0.0226016949011232 cubic meter 'market for concrete, normal' (cubic meter, BR, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>,
 Exchange: 0.000680925588410356 cubic meter 'market for concrete, normal' (cubic meter, CH, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>,
 Exchange: 0.00235506608812062 cubic meter 'market for concrete, normal' (cubic meter, CO, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>,
 Exchange: 0.01906591647549 cubic meter 'market for concrete, normal' (cubic meter, IN, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>,
 Exchange: 0.0107245780174631 cubic meter 'market for concrete, normal' (cubic meter, PE, None) to 'market group for concrete, normal' (cubic meter, GLO, None)>,
 Exchange: 0.0454155754069573 cubic

In [235]:
def change_market_shares(main_act, repl_act, market_share, method):
    
    gangnam_style = 1_000_000

    indices = np.array(
    [
        (gangnam_style, gangnam_style), # Production exchange for new main_act ( activity produces itself ) - need to have this to make matrix square
        (main_act.id, gangnam_style)]
        +
        [
        
        (exc.input.id, 1000) for exc in main_act.exchanges()
        #(sub_act.id, gangnam_style), # subtract sub_act - here we need to add all the sub_acts
    ] + [
        (node.id, gangnam_style) for node in [repl_act] # replace with this - something about new dps replacing old ones - could also have the values being added together?
        # see gh bw processing -> policies
    ], dtype=bwp.INDICES_DTYPE
    )
    
    data = np.array([
            1,
            1,
        ]
        +
        [
        exc['amount']*(10-market_share) for exc in list(main_act.exchanges())
        ]
        +
        [
            market_share # new activity - same amount
        ]
    ) 
    flip = np.array(
        [False, True] + [True for exc in list(main_act.exchanges())] + [True for _ in [repl_act]] # First False because ?, True - motor is consumed, False because numbers are negative ... ?,
        # WAIT I think I get it. This is the data array, where we put False for old act, and True for new act...
        # ...This is probably where we would want to change things to create new market shares...?
    ) # could alsos set minussign if you prefer but i still wouldnt know where
    
    # return(demand)
    
    dp = bwp.create_datapackage()

    dp.add_persistent_vector(
        matrix="technosphere_matrix",
        data_array=data,
        indices_array=indices,
        flip_array=flip,
        name="New technosphere",
    )
    
    _, data_objs, _ = bd.prepare_lca_inputs({main_act: 1}, ipcc) # data_objs - still using ei
    
    '''
    
    lca = bc.LCA({main_act.id: 1}, data_objs=data_objs + [dp]) # old motor + add [dp] (new motor)
    lca.lci()
    lca.lcia()
    lcascore1 = lca.score
    
    lca.lcia({gangnam_style: 1}) # new motor
    lcascore2 = lca.score
    
    '''
    
    # return(lcascore1, lcascore2)
    
    return(dp)

In [236]:
test_dp = change_market_shares(concrete_market, wood_chipping_act, 0.1, ipcc)

In [249]:
test_dp.metadata

{'profile': 'data-package',
 'name': 'c1592589a8f046e09d5fe58009312eda',
 'id': '7ce2092a948145dc841a1ce4953fe8ca',
 'licenses': [{'name': 'ODC-PDDL-1.0',
   'path': 'http://opendatacommons.org/licenses/pddl/',
   'title': 'Open Data Commons Public Domain Dedication and License v1.0'}],
 'resources': [{'profile': 'data-resource',
   'format': 'npy',
   'mediatype': 'application/octet-stream',
   'name': 'New technosphere.indices',
   'matrix': 'technosphere_matrix',
   'kind': 'indices',
   'path': 'New technosphere.indices.npy',
   'group': 'New technosphere',
   'category': 'vector',
   'nrows': 12},
  {'profile': 'data-resource',
   'format': 'npy',
   'mediatype': 'application/octet-stream',
   'name': 'New technosphere.data',
   'matrix': 'technosphere_matrix',
   'kind': 'data',
   'path': 'New technosphere.data.npy',
   'group': 'New technosphere',
   'category': 'vector',
   'nrows': 12},
  {'profile': 'data-resource',
   'format': 'npy',
   'mediatype': 'application/octet-stre

In [246]:
bwp.create_datapackage?

Signature:
bwp.create_datapackage(
    fs: Optional[fs.base.FS] = None,
    name: Optional[str] = None,
    id_: Optional[str] = None,
    metadata: Optional[dict] = None,
    combinatorial: bool = False,
    sequential: bool = False,
    seed: Optional[int] = None,
    sum_intra_duplicates: bool = True,
    sum_inter_duplicates: bool = False,
) -> bw_processing.datapackage.Datapackage
Docstring:
Create a new data package.

All arguments are optional; if a `PyFilesystem2 <https://docs.pyfilesystem.org/en/latest/>`__ filesystem is not provided, a `MemoryFS <https://docs.pyfilesystem.org/en/latest/reference/memoryfs.html>`__ will be used.

All metadata elements should follow the `datapackage specification <https://frictionlessdata.io/specs/data-package/>`__.

Licenses are specified as a list in ``metadata``. The default license is the `Open Data Commons Public Domain Dedication and License v1.0 <http://opendatacommons.org/licenses/pddl/>`__.

Args:

    * fs: A `Filesystem`, optional. A 

## 2) My first LCA

Brightway has a so-called LCA object.  
It is instantiated using `LCA(args)`.  
The only required argument is a functional unit, described by a dictionary with keys = activities and values = amounts (more [here](https://docs.brightwaylca.org/lca.html#specifying-a-functional-unit)).  
A second argument that is often passed is an LCIA method, passed using the method tuple.  

### 2.1) General syntax of LCA calculations

Let's create our first LCA object using our random activity and our IPCC method.  

In [ ]:
myFirstLCA_quick = bw.LCA({random_act:1}, ('IPCC 2013', 'climate change', 'GWP 100a'))

The steps to get to the impact score are as follows:

In [ ]:
myFirstLCA_quick.lci()    # Builds matrices, solves the system, generates an LCI matrix.
myFirstLCA_quick.lcia()   # Characterization, i.e. the multiplication of the elements 
                          # of the LCI matrix with characterization factors from the chosen method
myFirstLCA_quick.score    # Returns the score, i.e. the sum of the characterized inventory

Let's not take a closer look at the LCA object and its methods/attributes. We'll do this by creating a new LCA object: 

In [ ]:
myFirstLCA = bw.LCA({random_act:1}, ('IPCC 2013', 'climate change', 'GWP 100a'))

### 2.2) the `demand` attribute

In [ ]:
myFirstLCA.demand

To access the actual activity from the demand, you would do this:

In [ ]:
demanded_act = list(myFirstLCA.demand.keys())[0]
demanded_act

In [ ]:
demanded_act == random_act

There are also other attributes that have simply not been built yet, such as the `demand_array` and the `score`. To generate them, we first need to actually build the matrices. This will be done when calling the `.lci()` method.

### 2.3) Reminder of the system that needs to be solved in calculating an LCI

Before actually running the `.lci()` method, here's a quick refresher of the actual calculation that Brightway will need to do to calculate the inventory:  

$g=BA^{-1}f$  

where:  

  - $A$ = the technosphere matrix  
  - $B$ = the biosphere matrix (matrix with elementary flows)  
  - $f$ = the final demand vector  
  - $g$ = the inventory  

**Discussion:** Knowing what you do about the structure of Brightway (notably, activities and exchanges), what needs to happen to generate these matrices?

To consider:  
  - how should the order of the rows and columns be determined?  
  - how should we keep track of what is in each row and column?  
  - The parameters in the matrices are sometimes actually probability distribution functions - how should we consider this uncertainty information?  
  - The matrices are *sparse*, i.e. they are mostly made up of zeros. Should we consider this? Why? How?

### 2.4) Building the matrices

#### Structured arrays

LCI data imported in Brightway is stored in the `databases.db` database, discussed above.  
It is also stored in [numpy *structured arrays*](https://docs.scipy.org/doc/numpy/user/basics.rec.html). Here is a look at the  structured array for my ecoinvent 2.2 import (put in a pandas DataFrame because it looks nicer):  
<img src="images/structured_array_ecoinvent38.JPG">

**Exercise**(not core): Load the structured array of the ecoinvent database you are working with now.

In [ ]:
eidb.filepath_processed()

In [ ]:
your_structured_array = np.load(eidb.filepath_processed())
pd.DataFrame(your_structured_array).head()

In this array:  
  - `input` and `output` columns are integers that map to an activity. This mapping is found in the `mapping.pickle` file in the `project` directory and it looks something like this:

In [ ]:
pd.Series(bw.mapping).head()

  - `row` and `col`store *dummy* placeholder information about the location of the parameter in the matrices. 
  - the `type` indicates whether the exchange is a reference flow (`type`=0), technosphere exchange (`type`=1) or elementary flow (`type`=2).  
  - the other columns deal with uncertainty data. We'll cover that later, but one can always read about these columns in the [`stats_arrays` documentation](http://stats-arrays.readthedocs.io/en/latest/)

When the `.lci()` method is called, the structured arrayas are used to build matrices. The code responsible to do this is in the [`MatrixBuilder` class methods](https://2.docs.brightway.dev/technical/bw2calc.html#matrix-builder). 

The method `MatrixBuilder.build_dictionary` is used to take input and output values, respectively, and figure out which rows and columns they correspond to. The actual code is succinct - only one line - but what it does is:
  - Get all unique values, as each value will appear multiple times
  - Sort these values
  - Give them integer indices, starting with zero
This information on row and column indices is sufficient to build matrices. These matrices are build in a [COOrdinate sparse matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) format, where, for each exchange, three values are required: row position, column position, and amount (the actual value). The sparse matrices are actually stored in [CSR format](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix), but this is a detail.  

Some more details are are found [here](https://2.docs.brightway.dev/technical/bw2calc.html#matrix-builders).  

Let's now finally run the `.lci()`.

In [ ]:
myFirstLCA.lci()

Here's what the structured arrays *now* look like:  

In [ ]:
pd.DataFrame(myFirstLCA.bio_params).head(5) # Technosphere parameters are at myFirstLCA.tech_params

We see that the row and col numbers are no longer dummy variables, but that they actually have real matrix indices.

#### Dictionaries that map between incides and activities

One of the useful things that the `MatrixBuilder` produces are `dictionaries` that map row and column numbers to the keys of activities.  There are three such dictionaries, all directly accessible as attributes of the LCA object:
 - `activity_dict`: Columns in the technosphere matrix $A$ or biosphere matrix $B$
 - `product_dict` : Rows in the technosphere matrix $A$  
 - `biosphere_dict`: Rows in the biosphere matrix $B$

Here what this dictionary looks like:

In [ ]:
myFirstLCA.activity_dict

So, if I know the key to my activity (which, again,  is a `tuple` consisting of the database name and the activity code), I can read the column index (from `activity_dict`) or row index (from `product_dict` or `biosphere_dict` for the $A$ or $B$ matrices, respectively). 

Let's find out what column is associated with the activity that is producing our final demand as reference flow.

In [ ]:
# Getting the key from the `demand`attribute:
act_key = list(myFirstLCA.demand)[0].key
# Getting the column number from the activity_dict:
col_index = myFirstLCA.activity_dict[act_key]
print("The column index for activity {} is {}".format(act_key, col_index))

While this is useful, it is often more useful to determine what a row or column in the matrices actually refers to. In these cases, we need a dictionary that maps row or column indices to activity keys, and not the opposite.  
We can do this by reversing our dictionaries:

In [ ]:
myFirstLCA_rev_activity_dict = {value:key for key, value in myFirstLCA.activity_dict.items()}
myFirstLCA_rev_activity_dict

As a convenience, Brightway offers a method that will generate the three reverse dictionaries simultaneously.  
`.reverse_dict()` returns three reverse dicts (reverse activity dict,  reverse product dict,  reverse biosphere dict) *in that order*. The syntax for creating and assigning these reverse dicts is therefore: 

In [ ]:
myFirstLCA_rev_act_dict, myFirstLCA_rev_product_dict, myFirstLCA_rev_bio_dict = myFirstLCA.reverse_dict()

#### $A$ and $B$ Matrices

We can also access the matrices that were constructed. Let's look at the technosphere matrix ($A$).  
The ** $A$ matrix**, with each element $a_{ij}$ provides information on the amount of input or output of product $i$ from activity $j$. When $i=j$, the element $a_{ij}$ is the *reference flow* for the activity described in the column.

In [ ]:
myFirstLCA.technosphere_matrix

I am told that the dimensions of the matrix is $n*n$ where $n$ is the number of activities in my product system, and that the amount of actually stored elements is much less than $n^2$ (because the matrix is *sparse* and zero values are not stored).  

We can have an idea of what it stores by printing it out:

In [ ]:
print(myFirstLCA.technosphere_matrix)

It therefore stores both the coordinates and the values (as expected).
We can slice this matrix using coordinates. For example, let's say we wanted a view of the exchanges associated with the unit process providing our functional unit.  
We already know found the column number for that activity: 

In [ ]:
print("As a reminder, the column index for  {} is  {}".format(act_key, col_index))

To return the whole column from the $A$ matrix, we therefore slice the $A$ matrix.  
Python notes:  
  - In Python, slicing is done using []
  - We specify rows first, then columns  
  - `:` refers to "the whole row" or "the whole column" (depending if it is passed first or second in the []) 

In [ ]:
myColumn = myFirstLCA.technosphere_matrix[:, col_index]
myColumn

Printing this out gives:

In [ ]:
print(myColumn)

Not too useful: it would be better to get the *names* to these exchanges.  
We need to do two things:  
  - Get the indices from the CSR matrix (we can do this by converting it to a sparse matrix in `COOrdinate` format first)  
  - Get the activity code for the each index (we can do this using the reverse of the `activity_dict`)  
  - Use `get_activity` to access the actual names of the activities.  

1) Converting the CSR matrix to a COO matrix:  

In [ ]:
myColumnCOO = myColumn.tocoo()
myColumnCOO

It is still a sparse matrix with the same number of elements, and it looks quite like the CSR version when we print it out:

In [ ]:
print(myColumnCOO)

However, we can directly access the rows and column indices using `row` and `col`:  

In [ ]:
myColumnCOO.row

2) Get the activity code for each element using the **reverse** product dictionary we produced above:

In [ ]:
# Using a list comprehension:
[myFirstLCA_rev_product_dict[i] for i in myColumnCOO.row]

It would be even nicer to get the names for these:

In [ ]:
names_of_my_inputs = [bw.get_activity(myFirstLCA_rev_product_dict[i])['name'] for i in myColumnCOO.row]
names_of_my_inputs

We can put these in a neat Pandas Series, with actual names and amounts:

In [ ]:
# First create a dict with the information I want:
myColumnAsDict = dict(zip(names_of_my_inputs,myColumnCOO.data))
# Create Pandas Series from dict
pd.Series(myColumnAsDict, name="Nice series with information on exchanges in my foreground process")

Alternative way to generate similar information without even looking at the matrices:

In [ ]:
pd.Series({bw.get_activity(exc.input)['name']:exc.amount for exc in random_act.technosphere()}, 
          name="alternative way to generate exchanges")

Note the differences:  
  - The reference flow is not there (activity.technosphere() only returns technoshere exchanges where the input is not equal to the output)  
  - The values are positive, not negative (because the $A$ matrix is $I-Z$ where $Z$ contains the information on these inputs.

**Exercise**: Create a Pandas Series with the elementary flows of the activity supplying the reference flow for myFirstLCA.

In [ ]:
# Solution
myBioColumn = myFirstLCA.biosphere_matrix[:, col_index]
myBioValues = myBioColumn.tocoo().data
myBioNames = [bw.get_activity(myFirstLCA_rev_bio_dict[row])['name'] for row in myBioColumn.tocoo().row]
pd.Series(dict(zip(myBioNames, myBioValues)))

#### Demand array $f$

The demand array is the $f$ in $As=f$. 
It is an attribute of the LCA object:

In [ ]:
myFirstLCA.demand_array

Looks like it is all zeros, but not so:

In [ ]:
myFirstLCA.demand_array.sum()

So where is the one? We can know this by using our `activity_dict`.

In [ ]:
demand_database = list(myFirstLCA.demand.keys())[0]['database']
demand_code = list(myFirstLCA.demand.keys())[0]['code']
(demand_database, demand_code)

In [ ]:
row_of_demand = myFirstLCA.activity_dict[(demand_database, demand_code)]
row_of_demand # Row number of our demand vector containing the functional unit.

In [ ]:
myFirstLCA.demand_array[row_of_demand]

### 2.5) Solution to the inventory calculation

We saw above how `.lci()` produced the $A$ and $B$ matrices.  
`.lci()` also *solves* the equation $As=f$ and calculated the inventory by multiplying the solution to this equation by the biosphere matrix.  

#### Supply array

Vector containing the amount each activity will need to provide to meet the functional demand, i.e. $s=A^{-1}f$.

In [ ]:
myFirstLCA.supply_array

In [ ]:
myFirstLCA.supply_array.shape

**Inventory matrix**  
Contains the inventory *by activity* (i.e. not summed). In other words, we do not have $g=BA^{-1}f$, but rather $G=B \cdot diag(A^{-1}f)$

In [ ]:
myFirstLCA.inventory

We can aggregate the LCI results along the columns (i.e. calculate the cradle-to-gate inventory):

In [ ]:
LCI_cradle_to_gate = myFirstLCA.inventory.sum(axis=1)
LCI_cradle_to_gate.shape

**Exercise:** Get the total (cradle-to-gate) emissions of nitrous oxide emitted to air in the "urban air" subcompartment.

In [ ]:
NOx_act = [act for act in my_bio if 'Dinitrogen monoxide' in act['name']
                       and 'urban air close to ground' in str(act['categories'])
         ][0]
NOx_act.key

In [ ]:
NOx_row = myFirstLCA.biosphere_dict[NOx_act]
NOx_row

In [ ]:
myFirstLCA.inventory[NOx_row, :].sum()

### 2.7) LCIA

The LCIA calculation is done via the `.lcia()` method.

In [ ]:
myFirstLCA.lcia()

A number of other matrices are now available:

In [ ]:
# Matrix of characterization factors:
myFirstLCA.characterization_matrix

In [ ]:
myFirstLCA.characterization_matrix.shape

In [ ]:
# Matrix of characterized inventory flows
myFirstLCA.characterized_inventory

Question: would there be more, less or just as many elements in the inventory matrix as there are in the characterized inventory matrix?

The overall score is now an attribute of the LCA object: 

In [ ]:
myFirstLCA.score

We also could have determined what this score was by summing the elements of our `characterized_inventory` matrix:

In [ ]:
myFirstLCA.characterized_inventory.sum()

We could also have calculated it by multiplying the inventory and characterization factors ourselves:

In [ ]:
(myFirstLCA.characterization_matrix * myFirstLCA.inventory).sum()

We could also calculate the score by elementary flow (summing columns for each rows), irrespective of the unit process that produced it:

In [ ]:
elementary_flow_contribution = myFirstLCA.characterized_inventory.sum(axis=1) #Axis is the dimension I want to sum over:

In [ ]:
elementary_flow_contribution.shape

Notice that is has **two** dimensions. The result is in fact a one-dimensional matrix:

In [ ]:
type(elementary_flow_contribution)

To convert it to an array (probably more useful for many purposes), you can use any of the following approaches:

In [ ]:
elementary_flow_contribution.A1 
#np.squeeze(np.asarray(elementary_flow_contribution))
#np.asarray(elementary_flow_contribution).reshape(-1)
#np.array(elementary_flow_contribution).flatten()
#np.array(elementary_flow_contribution).ravel()

**Exercise:** Create a Pandas series that has the scores per unit process, sorted by value (contribution analysis)

In [ ]:
# Create array with the results per column (i.e. per activity)
results_by_activity = (myFirstLCA.characterized_inventory.sum(axis=0)).A1

In [ ]:
# Create a list of names in columns
list_of_names_in_columns = [bw.get_activity(myFirstLCA_rev_act_dict[col])['name'] 
                            for col in range(myFirstLCA.characterized_inventory.shape[1])]

In [ ]:
pd.Series(index=list_of_names_in_columns, data=results_by_activity).sort_values(ascending=False).head(10)

## 3) My second LCA: Comparative LCA

Let's choose two activities to compare, say Swiss electricity produced from respectively a run-of-river hydropower plant and a wind turbine.

**Exercise**: assign the two activities to variables `hydro` and `wind` respectively.

In [ ]:
[act for act in eidb if "wind" in act['name'] and "electricity" in act['name'] and "CH" in act['location']]

In [ ]:
wind = [act for act in eidb if "wind" in act['name']
        and "<1MW" in act['name'] 
        and "kilowatt hour" in act['unit']][0]
wind

In [ ]:
[act for act in eidb if "hydro" in act['name'] and "river" in act['name'] and "CH" in act['location']]

In [ ]:
hydro = [act for act in eidb if "hydro" in act['name'] 
                     and "river" in act['name'] 
                     and "CH" in act['location']
                     and "kilowatt hour" in act['unit']
                     ][0]
hydro

Let's also compare these according to their carbon footprint as measured with the IPCC method we already selected above:

In [ ]:
ipcc_2013_method

#### One at a time approach:

In [ ]:
hydroLCA = bw.LCA({hydro:1}, ipcc_2013_method.name)
hydroLCA.lci()
hydroLCA.lcia()
hydroLCA.score

**Exercise:** Do the LCA for `wind`:

In [ ]:
windLCA = bw.LCA({wind:1}, ipcc_2013_method.name)
windLCA.lci()
windLCA.lcia()
windLCA.score

In [ ]:
#Compare results:
if windLCA.score>hydroLCA.score:
    print("Hydro is preferable")
elif windLCA.score<hydroLCA.score:
    print("Wind is preferable")
else:
    print("Both options have the same climate change indicator result")

Do one "delta" LCA:

In [ ]:
deltaLCA = bw.LCA({wind:1, hydro:-1}, ipcc_2013_method.name)
deltaLCA.lci()
deltaLCA.lcia()
deltaLCA.score

In [ ]:
if deltaLCA.score>0:
    print("Hydro is preferable")
elif deltaLCA.score<0:
    print("Wind is preferable")
else:
    print("Both options have the same climate change indicator result")

## 4) My third LCA - Multiple impact categories

Say we want to evaluate the indicator results for our randomAct for all EF v3 categories (with long-term emissions).

In [ ]:
# Make a list of all impact method names (tuples):
EFV3 = [method for method in bw.methods if "EF v3.0" in str(method) 
        and "no LT" not in str(method)
        and "EN15804" not in str(method)]
EFV3

Simplest way: for loop, using `switch method`

In [ ]:
myThirdLCA = bw.LCA({random_act:1}, EFV3[0]) # Do LCA with one impact category
myThirdLCA.lci()
myThirdLCA.lcia()
for category in EFV3:
    myThirdLCA.switch_method(category)
    myThirdLCA.lcia()
    print("Score is {:f} {} for category {}".format(myThirdLCA.score, 
                                                 bw.Method(category).metadata['unit'],
                                                 bw.Method(category).name)
          )

In [ ]:
myFirstLCA_unitProcessContribution = myFirstLCA.characterized_inventory.sum(axis=0).A1
myFirstLCA_unitProcessRelativeContribution = myFirstLCA_unitProcessContribution/myFirstLCA.score

## Revising my second and third LCA with `MultiLCA`

The `MultiLCA` allows thecalculation of LCA results for multiple functional units and impact categories.  
One simply needs to create a `calculation setup`, i.e. a named set of functional units and LCIA methods.

Calculation setups: dictionary with lists of functional units and methods.

In [ ]:
list_functional_units = [{wind.key:1}, {hydro.key:1}]
list_methods = EFV3

In [ ]:
bw.calculation_setups['wind_vs_hydro'] = {'inv':list_functional_units, 'ia':list_methods}

In [ ]:
bw.calculation_setups['wind_vs_hydro']

In [ ]:
myMultiLCA = bw.MultiLCA('wind_vs_hydro')

In [ ]:
myMultiLCA.results.shape

In [ ]:
myMultiLCA.results

In [ ]:
pd.DataFrame(index=EFV3, columns=[wind['name'], hydro['name']], data=myMultiLCA.results.T)

You can also create "fuller" DataFrames. Here is with code from [here](http://stackoverflow.com/questions/42984831/create-a-dataframe-from-multilca-results-in-brightway2): 

In [ ]:
scores = pd.DataFrame(myMultiLCA.results, columns=myMultiLCA.methods)
as_activities = [
    (bw.get_activity(key), amount) 
    for dct in myMultiLCA.func_units 
    for key, amount in dct.items()
]
nicer_fu = pd.DataFrame(
    [
        (x['database'], x['code'], x['name'], x['location'], x['unit'], y) 
        for x, y in as_activities
    ], 
    columns=('Database', 'Code', 'Name', 'Location', 'Unit', 'Amount')
)
pd.concat([nicer_fu, scores], axis=1).T

You can even generate beautiful heatmaps like this in a relatively easy way, see example notebook [here](https://github.com/brightway-lca/brightway2/blob/master/notebooks/Using%20calculation%20setups.ipynb).

<img src="images/multiLCA_heatmap.JPG">

Done with deterministic LCA using only existing database items!